<a href="https://colab.research.google.com/github/Yi-Wei-Lin/Tibame_ML_20210602/blob/main/%E9%A6%99%E8%95%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [221]:
import pandas as pd
train_df = pd.read_csv("test.csv", encoding="utf-8")

# Date 民國 to 西元
def date_df(d):
  d = d.split("/")
  year = str(int(d[0])+1911)
  month = (d[1])
  day = (d[2])
  d = (f"{year}-{month}-{day}")
  return d
date_df = pd.DataFrame({"Date":train_df["Date"].apply(date_df)})
train_df.update(date_df)

# Month method
def Month(m):
  Month_data = int(m.split("-")[1])
  return Month_data
train_df["Month"] = train_df["Date"].apply(Month)

# Week_day method
def week_day(d):
  d = d.split("-")
  year = int(d[0])
  month = int(d[1])
  day = int(d[2])
  temp = pd.Timestamp(f"{year}-{month}-{day}")
  week_day = temp.dayofweek + 1 #temp.dayofweek 0(禮拜一) 開始, +1是因為比較直接禮拜幾
  return week_day
train_df["Week_day"] = train_df["Date"].apply(week_day)
train_df["Year"] = train_df["Date"].apply(lambda d:int(d.split("-")[0]))

# Rest_market
market_df = pd.DataFrame({"Market":train_df["Market"].apply(lambda t:t.split(" ")[1])})
train_df.update(market_df)

# Volume chnage from str to int
train_df["Volume"] = train_df["Volume"].apply(lambda x:int(x.replace(",","")))

# Up_price change from onj to float
# train_df["Up_price"] = train_df["Up_price"].apply(lambda x:float(x))

train_df

,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Volume,Month,Week_day,Year
0,2001-01-01,¥x¥_¤G,A1 ­»¿¼,35.0,30.0,26.7,30.4,2679,1,1,2001
1,2001-01-02,¥x¥_¤G,A1 ­»¿¼,32.9,25.5,22.0,26.3,4589,1,2,2001
2,2001-01-03,¥x¥_¤G,A1 ­»¿¼,32.7,26.4,22.0,26.8,5437,1,3,2001
3,2001-01-05,¥x¥_¤G,A1 ­»¿¼,32.5,24.6,19.7,25.2,7835,1,5,2001
4,2001-01-06,¥x¥_¤G,A1 ­»¿¼,29.8,24.1,19.2,24.3,5896,1,6,2001
...,...,...,...,...,...,...,...,...,...,...,...
56437,2021-06-12,¥xªF¥«,A1 ­»¿¼,32.0,21.5,15.0,22.8,3007,6,6,2021
56438,2021-06-13,¥xªF¥«,A1 ­»¿¼,30.0,26.2,18.0,27.9,3558,6,7,2021
56439,2021-06-14,¥xªF¥«,A1 ­»¿¼,30.0,26.2,18.0,22.9,2857,6,1,2021
56440,2021-06-17,¥xªF¥«,A1 ­»¿¼,33.0,22.9,15.0,20.8,3052,6,4,2021


In [ ]:
train_df["Date"] = pd.to_datetime(train_df["Date"])
train_df["Date"]

0       2001-01-01
1       2001-01-02
2       2001-01-03
3       2001-01-05
4       2001-01-06
           ...    
56452   2021-06-12
56453   2021-06-13
56454   2021-06-14
56455   2021-06-17
56456   2021-06-19
Name: Date, Length: 56457, dtype: datetime64[ns]

In [ ]:
train_df
train_df.dtypes

Date         datetime64[ns]
Market               object
Product              object
Up_price             object
Mid_price           float64
Low_price           float64
Avg_price           float64
Volume                int64
Month                 int64
Week_day              int64
Year                  int64
dtype: object

In [ ]:
a = train_df["Market"] == "台北二"
t_index = pd.date_range(train_df[a]["Date"][0], '2021-06-19', freq='D') #start_time by 市場別
T_df = pd.DataFrame(t_index, columns=['Date'])
m = train_df[a].merge(T_df, how="right",sort=True)
most = m.value_counts().idxmax()
m["Market"]=m["Market"].fillna(most[1])
m["Product"]=m["Product"].fillna(most[2])

na = m["Volume"].isna()
m["Month"] = m["Date"].apply(lambda x:int(str(x).split("-")[1]))
m["Week_day"] = m["Date"].apply(lambda x:pd.Timestamp(str(x).split()[0]).dayofweek+1)
m["Year"] = m["Date"].apply(lambda x:int(str(x).split("-")[0]))
import math
def restday(s):
  if s > 0:
    return 0
  else:
    return 1
m["Rest_day"] = m["Volume"].apply(restday)
print(m.dtypes)
m

Date         datetime64[ns]
Market               object
Product              object
Up_price             object
Mid_price           float64
Low_price           float64
Avg_price           float64
Volume              float64
Month                 int64
Week_day              int64
Year                  int64
Rest_day              int64
dtype: object


,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Volume,Month,Week_day,Year,Rest_day
0,2001-01-01,台北二,A1 香蕉,35,30.0,26.7,30.4,2679.0,1,1,2001,0
1,2001-01-02,台北二,A1 香蕉,32.9,25.5,22.0,26.3,4589.0,1,2,2001,0
2,2001-01-03,台北二,A1 香蕉,32.7,26.4,22.0,26.8,5437.0,1,3,2001,0
3,2001-01-04,台北二,A1 香蕉,NaN,NaN,NaN,NaN,NaN,1,4,2001,1
4,2001-01-05,台北二,A1 香蕉,32.5,24.6,19.7,25.2,7835.0,1,5,2001,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7470,2021-06-15,台北二,A1 香蕉,NaN,NaN,NaN,NaN,NaN,6,2,2021,1
7471,2021-06-16,台北二,A1 香蕉,NaN,NaN,NaN,NaN,NaN,6,3,2021,1
7472,2021-06-17,台北二,A1 香蕉,36.8,28.7,20.5,28.6,10080.0,6,4,2021,0
7473,2021-06-18,台北二,A1 香蕉,39.1,28.5,17.0,28.3,8045.0,6,5,2021,0


In [ ]:
老師不好意思, 想詢問一下專題上遇到的問題, 第一張圖是專題大概的簡介(農產品價格預測), 第二章圖是目前遇到的困難, 
由於目前 input data 有很多批發市場(~13個左右), 每個市場都有不同的價格, 之前聽宇春老師的意思好像可以全部 data
都丟進去 model 裡面 training, 可是每種批發價格都不一樣, 這樣預測的價格不會有問題嗎, 還是我有 13 個批發市場
我可以 output 13個批發市場的預測價格呢??, 我有點不太懂, 所以想詢問一下你的建議, 感謝.
另外想問一下 13個批發市場每天不一定都有開市, 有些市場與其他市場休市時間不一樣, 假設110/06/06 有13個開市13個價格
跟 110/06/07 有10個開市10個價格, 這樣 input data 會不會有問題, 是否要補齊其他3個休市的缺失值呢??
然後氣候特徵工程的補缺失值得部分, 像是我有一些氣象資料有缺可能某些縣市會缺很多 data, 這種是要靠鄰近的縣市補強, 還是就直接省去呢??